In [0]:
# Downloading and Unzipping data in google colaboratory runtime files
!wget "https://datahack-prod.s3.amazonaws.com/train_file/train_aox2Jxw.zip" -O train.zip; unzip train.zip; rm train.zip

--2019-04-20 21:41:31--  https://datahack-prod.s3.amazonaws.com/train_file/train_aox2Jxw.zip
Resolving datahack-prod.s3.amazonaws.com (datahack-prod.s3.amazonaws.com)... 52.219.62.4
Connecting to datahack-prod.s3.amazonaws.com (datahack-prod.s3.amazonaws.com)|52.219.62.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8087338 (7.7M) [application/zip]
Saving to: ‘train.zip’

train.zip           100%[===================>]   7.71M  3.69MB/s    in 2.1s    

2019-04-20 21:41:34 (3.69 MB/s) - ‘train.zip’ saved [8087338/8087338]

Archive:  train.zip
  inflating: train.csv               
   creating: __MACOSX/
  inflating: __MACOSX/._train.csv    
  inflating: Data Dictionary.xlsx    
  inflating: __MACOSX/._Data Dictionary.xlsx  


In [0]:
import pandas as pd

In [0]:
train = pd.read_csv("train.csv")
test  = pd.read_csv("test_bqCt9Pv.csv") #uploaded to Google Colab directly

In [0]:
# Looking at the data headers, these values aren't required
train = train.drop(['UniqueID', 'supplier_id', 'Current_pincode_ID', 'Date.of.Birth', 'DisbursalDate', 'Employee_code_ID'], axis = 1)
test = test.drop(['UniqueID', 'supplier_id', 'Current_pincode_ID', 'Date.of.Birth', 'DisbursalDate', 'Employee_code_ID'], axis = 1)

In [0]:
print(train.shape)
print(test.shape)

(233154, 35)
(112392, 34)


In [0]:
Y = train.iloc[:, -1] #last column is the the prediction in the training set

In [0]:
Y.shape

(233154,)

In [0]:
X = train.drop(['loan_default'], axis = 1)

In [0]:
X.shape

(233154, 34)

In [0]:
test_X = test.iloc[:,:]

In [0]:
X.sample(3) # Checking whether irrelevant rows are dropped or not

,disbursed_amount,asset_cost,ltv,branch_id,manufacturer_id,Employment.Type,State_ID,MobileNo_Avl_Flag,Aadhar_flag,PAN_flag,...,SEC.CURRENT.BALANCE,SEC.SANCTIONED.AMOUNT,SEC.DISBURSED.AMOUNT,PRIMARY.INSTAL.AMT,SEC.INSTAL.AMT,NEW.ACCTS.IN.LAST.SIX.MONTHS,DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS,AVERAGE.ACCT.AGE,CREDIT.HISTORY.LENGTH,NO.OF_INQUIRIES
193525,52103,71309,75.73,16,86,Salaried,14,1,1,0,...,0,0,0,0,0,0,0,0yrs 0mon,0yrs 0mon,0
110739,46000,59175,79.43,72,45,Self employed,11,1,1,0,...,0,0,0,16954,0,0,1,2yrs 5mon,6yrs 6mon,0
187420,22169,39482,59.93,5,48,Salaried,9,1,1,0,...,0,0,0,0,0,0,0,0yrs 0mon,0yrs 0mon,0


In [0]:
X['Employment.Type'].fillna('Self employed', inplace = True)
test_X['Employment.Type'].fillna('Self employed', inplace = True)

In [0]:
X['Employment.Type'].value_counts()

Self employed    135296
Salaried          97858
Name: Employment.Type, dtype: int64

In [0]:
X['Employment.Type'] = X['Employment.Type'].replace(('Unemployed', 'Salaried', 'Self employed'), (0, 1, 2))
test_X['Employment.Type'] = test_X['Employment.Type'].replace(('Unemployed', 'Salaried', 'Self employed'), (0, 1, 2))

In [0]:
X['Employment.Type'].value_counts() #Converted irrelevant strings to numbers for computations while training

2    135296
1     97858
Name: Employment.Type, dtype: int64

In [0]:
X['PERFORM_CNS.SCORE.DESCRIPTION'].value_counts()

No Bureau History Available                                116950
C-Very Low Risk                                             16045
A-Very Low Risk                                             14124
D-Very Low Risk                                             11358
B-Very Low Risk                                              9201
M-Very High Risk                                             8776
F-Low Risk                                                   8485
K-High Risk                                                  8277
H-Medium Risk                                                6855
E-Low Risk                                                   5821
I-Medium Risk                                                5557
G-Low Risk                                                   3988
Not Scored: Sufficient History Not Available                 3765
J-High Risk                                                  3748
Not Scored: Not Enough Info available on the customer        3672
Not Scored

In [0]:
X['PERFORM_CNS.SCORE.DESCRIPTION'] = X['PERFORM_CNS.SCORE.DESCRIPTION'].replace(('No Bureau History Available', 
                                     'Not Scored: Sufficient History Not Available','Not Scored: Not Enough Info available on the customer',
                                     'Not Scored: No Activity seen on the customer (Inactive)', 
                                     'Not Scored: No Updates available in last 36 months', 'Not Scored: Only a Guarantor', 
                                     'Not Scored: More than 50 active Accounts found'),(0, 0, 0, 0, 0, 0, 0))

X['PERFORM_CNS.SCORE.DESCRIPTION'] = X['PERFORM_CNS.SCORE.DESCRIPTION'].replace(('L-Very High Risk', 'M-Very High Risk'), (1, 1))

X['PERFORM_CNS.SCORE.DESCRIPTION'] = X['PERFORM_CNS.SCORE.DESCRIPTION'].replace(('J-High Risk', 'K-High Risk'), (2, 2))

X['PERFORM_CNS.SCORE.DESCRIPTION'] = X['PERFORM_CNS.SCORE.DESCRIPTION'].replace(('H-Medium Risk', 'I-Medium Risk'), (3, 3))

X['PERFORM_CNS.SCORE.DESCRIPTION'] = X['PERFORM_CNS.SCORE.DESCRIPTION'].replace(('E-Low Risk', 'F-Low Risk', 'G-Low Risk'), (4, 4, 4))

X['PERFORM_CNS.SCORE.DESCRIPTION'] = X['PERFORM_CNS.SCORE.DESCRIPTION'].replace(('A-Very Low Risk', 'B-Very Low Risk',
                                      'C-Very Low Risk', 'D-Very Low Risk'), (5, 5, 5, 5))

In [0]:
X['PERFORM_CNS.SCORE.DESCRIPTION'].value_counts()

0    129785
5     50728
4     18294
3     12412
2     12025
1      9910
Name: PERFORM_CNS.SCORE.DESCRIPTION, dtype: int64

In [0]:
test_X['PERFORM_CNS.SCORE.DESCRIPTION'].value_counts()

No Bureau History Available                                57893
C-Very Low Risk                                             7825
A-Very Low Risk                                             7559
D-Very Low Risk                                             5114
B-Very Low Risk                                             4495
M-Very High Risk                                            4032
K-High Risk                                                 3864
F-Low Risk                                                  3691
H-Medium Risk                                               3287
I-Medium Risk                                               2703
E-Low Risk                                                  2572
G-Low Risk                                                  1807
J-High Risk                                                 1778
Not Scored: No Activity seen on the customer (Inactive)     1361
Not Scored: Not Enough Info available on the customer       1268
Not Scored: Sufficient Hi

In [0]:
test_X['PERFORM_CNS.SCORE.DESCRIPTION'] = test_X['PERFORM_CNS.SCORE.DESCRIPTION'].replace(('No Bureau History Available', 
                                     'Not Scored: Sufficient History Not Available','Not Scored: Not Enough Info available on the customer',
                                     'Not Scored: No Activity seen on the customer (Inactive)', 
                                     'Not Scored: No Updates available in last 36 months', 'Not Scored: Only a Guarantor', 
                                     'Not Scored: More than 50 active Accounts found'),(0, 0, 0, 0, 0, 0, 0))

test_X['PERFORM_CNS.SCORE.DESCRIPTION'] = test_X['PERFORM_CNS.SCORE.DESCRIPTION'].replace(('L-Very High Risk', 'M-Very High Risk'), (1, 1))

test_X['PERFORM_CNS.SCORE.DESCRIPTION'] = test_X['PERFORM_CNS.SCORE.DESCRIPTION'].replace(('J-High Risk', 'K-High Risk'), (2, 2))

test_X['PERFORM_CNS.SCORE.DESCRIPTION'] = test_X['PERFORM_CNS.SCORE.DESCRIPTION'].replace(('H-Medium Risk', 'I-Medium Risk'), (3, 3))

test_X['PERFORM_CNS.SCORE.DESCRIPTION'] = test_X['PERFORM_CNS.SCORE.DESCRIPTION'].replace(('E-Low Risk', 'F-Low Risk', 'G-Low Risk'), (4, 4, 4))

test_X['PERFORM_CNS.SCORE.DESCRIPTION'] = test_X['PERFORM_CNS.SCORE.DESCRIPTION'].replace(('A-Very Low Risk', 'B-Very Low Risk',
                                      'C-Very Low Risk', 'D-Very Low Risk'), (5, 5, 5, 5))
 

In [0]:
test_X['PERFORM_CNS.SCORE.DESCRIPTION'].value_counts()

0    63017
5    24993
4     8070
3     5990
2     5642
1     4680
Name: PERFORM_CNS.SCORE.DESCRIPTION, dtype: int64

In [0]:
import re
def toMonths(str):
  cache = []
  for k in X[str]:
    temp = int(re.split("[yrs mon]+", k)[0]) * 12 + int(re.split("[yrs mon]+", k)[1])
    cache.append(temp)
  return cache

In [0]:
def toMonthstest(str):
  cache = []
  for k in test_X[str]:
    temp = int(re.split("[yrs mon]+", k)[0]) * 12 + int(re.split("[yrs mon]+", k)[1])
    cache.append(temp)
  return cache

In [0]:
X['CREDIT.HISTORY.LENGTH'] = toMonths('CREDIT.HISTORY.LENGTH')
X['CREDIT.HISTORY.LENGTH'][:5]

0     0
1    23
2     0
3    15
4     0
Name: CREDIT.HISTORY.LENGTH, dtype: int64

In [0]:
X['AVERAGE.ACCT.AGE'] = toMonths('AVERAGE.ACCT.AGE')

In [0]:
X['AVERAGE.ACCT.AGE'][:5]

0     0
1    23
2     0
3     8
4     0
Name: AVERAGE.ACCT.AGE, dtype: int64

In [0]:
test_X['CREDIT.HISTORY.LENGTH'] = toMonthstest('CREDIT.HISTORY.LENGTH')
test_X['AVERAGE.ACCT.AGE'] = toMonthstest('AVERAGE.ACCT.AGE')
test_X['AVERAGE.ACCT.AGE'][0:5]

0     0
1     8
2     0
3    29
4     0
Name: AVERAGE.ACCT.AGE, dtype: int64

In [0]:
from imblearn.over_sampling import SMOTE

x_train, y_train = SMOTE().fit_sample(X, Y.values.ravel()) 

In [0]:
print(x_train.shape)
print(y_train.shape)

(365086, 34)
(365086,)


In [0]:
import numpy as np
from sklearn.model_selection import train_test_split

#splitting training data into train and validation set
X_train, X_valid, Y_train, Y_valid = train_test_split(x_train, y_train, test_size = 0.2, random_state = 0)

In [0]:
print(X_train.shape)
print(Y_train.shape)

(292068, 34)
(292068,)


In [0]:
print(X_valid.shape)
print(Y_valid.shape)

(73018, 34)
(73018,)


In [0]:
from sklearn.preprocessing import StandardScaler

scalar = StandardScaler()
X_train = scalar.fit_transform(X_train)
X_valid = scalar.transform(X_valid)
test_X = scalar.transform(test_X)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  


In [0]:
from xgboost.sklearn import XGBClassifier
from sklearn.metrics import roc_auc_score

modelXG = XGBClassifier()
modelXG.fit(X_train, Y_train)

Y_predXG = modelXG.predict(X_valid)

print("Train Accuracy: ", modelXG.score(X_train, Y_train))
print("Validation Accuracy: ", modelXG.score(X_valid, Y_valid))

print("AUROC Score of XGBoost = ", roc_auc_score(Y_valid, Y_predXG))

Train Accuracy:  0.8288412287549475
Validation Accuracy:  0.8272069900572462
AUROC Score of XGBoost =  0.8265938225237444


In [0]:
from sklearn.ensemble import RandomForestClassifier

modelRF = RandomForestClassifier()
modelRF.fit(X_train, Y_train)

Y_predRF = modelRF.predict(X_valid)

print("Train Accuracy: ", modelRF.score(X_train, Y_train))
print("Validation Accuracy: ", modelRF.score(X_valid, Y_valid))

print("AUROC Score of Random Forest = ", roc_auc_score(Y_valid, Y_predRF))

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Train Accuracy:  0.9834901461303532
Validation Accuracy:  0.8365608480100797
AUROC Score of Random Forest =  0.8362252350232474


In [0]:
from sklearn.ensemble import AdaBoostClassifier

modelAB = AdaBoostClassifier()
modelAB.fit(X_train, Y_train)

Y_predAB = modelAB.predict(X_valid)

print("Train Accuracy: ", modelAB.score(X_train, Y_train))
print("Validation Accuracy: ", modelAB.score(X_valid, Y_valid))

print("AUROC Score of AdaBoost = ", roc_auc_score(Y_valid, Y_predAB))

Train Accuracy:  0.7895113466726926
Validation Accuracy:  0.7885726807088663
AUROC Score of AdaBoost =  0.7881084133400471


In [0]:
from sklearn.ensemble import GradientBoostingClassifier

modelGB = GradientBoostingClassifier()
modelGB.fit(X_train, Y_train)

Y_predGB = modelGB.predict(X_valid)

print("Training Accuracy: ", modelGB.score(X_train, Y_train))
print('Testing Accuarcy: ', modelGB.score(X_valid, Y_valid))

print("AUROC Score of Gradient Boosting = ", roc_auc_score(Y_valid, Y_predGB))

Training Accuracy:  0.8296492597614255
Testing Accuarcy:  0.827850666958832
AUROC Score of Gradient Boosting =  0.8272430477219446


In [0]:
from lightgbm import LGBMClassifier

modelLGBM = LGBMClassifier()
modelLGBM.fit(X_train, Y_train)

Y_predLGBM = modelLGBM.predict(X_valid)

print("Training Accuracy: ", modelLGBM.score(X_train, Y_train))
print('Testing Accuarcy: ', modelLGBM.score(X_valid, Y_valid))

print("AUROC Score of LGBM = ", roc_auc_score(Y_valid, Y_predLGBM))

Training Accuracy:  0.8525172220167906
Testing Accuarcy:  0.8504204442740146
AUROC Score of LGBM =  0.8498838616255058


In [0]:
test_Y_RF = modelRF.predict(test_X)
test_Y_XG = modelXG.predict(test_X)
test_Y_AB = modelAB.predict(test_X)
test_Y_LGBM = modelLGBM.predict(test_X)
test_Y_GB = modelGB.predict(test_X)
test_Y_pred = []

for i in range(len(test_Y_RF)):
  k = 0.35 * test_Y_LGBM[i] + 0.25 * test_Y_RF[i] + 0.175 * test_Y_GB[i] + 0.125 * test_Y_XG[i] + 0.1 * test_Y_AB[i] # weighted averaging
  test_Y_pred.append(k)

In [0]:
output = pd.read_csv('sample_submission_24jSKY6.csv')

output.head()

,UniqueID,loan_default
0,655269,0
1,723482,0
2,758529,0
3,763449,0
4,708663,0


In [0]:
test = pd.read_csv('test_bqCt9Pv.csv')
uid = test['UniqueID']
output = pd.DataFrame({'UniqueID': uid, 'loan_default': test_Y_pred})

output.head()

,UniqueID,loan_default
0,655269,0.3
1,723482,0.0
2,758529,0.0
3,763449,0.0
4,708663,0.0


In [0]:
output.to_csv('final.csv',index=False)

In [0]:
from google.colab import files
files.download("final.csv")